This notebook lays out instructions on how data and variables which went through different processing procedures are merged 
###  Follow these steps to merge and aggregate data and variables
1. Load ad metadata table (the masterfile, queried in the data paration step) that contains metadata for all ad types, such as ad_url, advertiser_id, impressions, etc.
2. Add content variables for TEXT ads ('ad_title', 'ad_text') and remove duplicates 
3. Add variables particular to VIDEO ads ('google_asr_text', 'aws_ocr_video_text', 'aws_face_vid')
4. Add variables particular to IMAGE ads ('aws_ocr_img_text', 'aws_face_vid')

### Step 1: Load ad metadata table (ad_id level masterfile)

In [2]:
import os
import pandas as pd

In [15]:
my_metadata_filepath = ''

In [12]:
df = pd.read_csv(my_metadata_filepath)

In [20]:
'''
Preview of the ad metadata table prepared from the repo image-video-data-preparation
Contains all ad types: TEXT, VIDEO, IMAGE
'''
print(df.columns)
df.head(4)

Index(['index', 'ad_id', 'ad_url', 'ad_type', 'advertiser_id',
       'advertiser_name', 'date_range_start', 'date_range_end', 'num_of_days',
       'impressions', 'age_targeting', 'gender_targeting',
       'geo_targeting_included', 'geo_targeting_excluded',
       'spend_range_min_usd', 'spend_range_max_usd'],
      dtype='object')


,index,ad_id,ad_url,ad_type,advertiser_id,advertiser_name,date_range_start,date_range_end,num_of_days,impressions,age_targeting,gender_targeting,geo_targeting_included,geo_targeting_excluded,spend_range_min_usd,spend_range_max_usd
0,27953,CR00000257354440376321,https://adstransparency.google.com/advertiser/...,VIDEO,AR08588079303567081473,Abrams For Governor,2022-08-03,2022-09-13,41,400000-450000,"18-24, 25-34, 35-44, 45-54, 55-64, ≥65","Male, Female, Unknown gender","30008,Georgia,United States, 30012,Georgia,Uni...","Alabama,United States, Florida,United States, ...",5000,6000
1,12080,CR00000354386341527553,https://adstransparency.google.com/advertiser/...,TEXT,AR03715945093920718849,PROGRESSNOW COLORADO,2022-11-07,2022-11-08,3,0-1000,NaN,NaN,"80120,Colorado,United States, 80121,Colorado,U...","Arizona,United States, California,United State...",0,100
2,7192,CR00000387990165651457,https://adstransparency.google.com/advertiser/...,TEXT,AR02250229023886016513,Mo Kashmiri for Sacramento County Office of Ed...,2022-05-15,2022-06-09,27,2000-3000,NaN,NaN,"95811,California,United States, 95814,Californ...",NaN,500,600
3,51899,CR00000440354406924289,https://adstransparency.google.com/advertiser/...,IMAGE,AR11849446020951834625,YANG FOR NEW YORK INC,2021-05-13,2021-06-07,25,1000-2000,NaN,NaN,"Brooklyn,New York,United States","11204,New York,United States, 11219,New York,U...",0,100


### Step 2: Add content variables for TEXT ads and remove duplicates

In [16]:
text = pd.read_csv(source_path + "01_prepare_ad_media/g2022_ad_text.csv")

In [21]:
'''
Preview of text ad data variables
'''

print(text.columns)
text.head(4)

Index(['ad_id', 'ad_title', 'ad_text', 'url', 'all_urls'], dtype='object')


,ad_id,ad_title,ad_text,url,all_urls
0,CR00222350456913920001,Paul Tonko For Congress,Paul doesn’t care about partisan games. He is ...,secure.actblue.com/,NaN
1,CR00208584571334164481,NaN,NaN,https://adstransparency.google.com/advertiser/...,NaN
2,CR00004163369498050561,NaN,NaN,https://adstransparency.google.com/advertiser/...,NaN
3,CR00009722500288086017,NaN,NaN,https://adstransparency.google.com/advertiser/...,NaN


In [24]:
'''
Duplicates clearly exist in text data, for example:
'''

text.loc[text.duplicated(subset=['ad_id'], keep=False)].sort_values(by='ad_id').head(4)

,ad_id,ad_title,ad_text,url,all_urls
8565,CR00000501995777556481,NaN,1 Message Pending\nView Pending Messages\nNow,https://adstransparency.google.com/advertiser/...,https://theblueballot.com/stim/
8564,CR00000501995777556481,NaN,1 Message Pending\nView Pending Messages\nNow,https://adstransparency.google.com/advertiser/...,https://theblueballot.com/stim/
96089,CR00001341747783270401,NaN,Who's on my ballot?\nThe 2022 Maine\nIndepende...,https://adstransparency.google.com/advertiser/...,https://join.goodparty.org/voter-guide/?utm_so...
96090,CR00001341747783270401,NaN,Who's on my ballot?\nThe 2022 Maine\nIndepende...,https://adstransparency.google.com/advertiser/...,https://join.goodparty.org/voter-guide/?utm_so...


In [ ]:
'''
Step 2.1: Drop duplicates based on all variables of the current table
'''
text.drop_duplicates(inplace=True)

In [ ]:
'''
Step 2.2: Check ad url variable and keep the first value of duplicates
'''
text = text.drop_duplicates(subset=['ad_id'], keep='first')

In [ ]:
'''
Step 2.3: 

Merge these variables that contain text information (and additional url info) for text ads into the metadata table:

'ad_id', 'ad_title', 'ad_text', 'all_urls' 
'''

df = df.merge(text[['ad_id', 'ad_title', 'ad_text', 'all_urls']], how='left', on='ad_id')

### Step 3: Merge variables derived from video processing

Merge video analysis results obtained from the **aws-rekognition-image-video-processing** repo

Merge these variables into the metadata table: 

+ 'google_asr_text'
+ 'aws_ocr_video_text'
+ 'aws_face_vid'


### Step 4: Merge variables derived from image processing

Merge image analysis results obtained from the **aws-rekognition-image-video-processing** repo

+ 'aws_ocr_img_text'
+ 'aws_face_vid'


### Step 5: Save final output table at the ad_id level

In [ ]:
outfilepath = 'outfile.csv'
df.to_csv(outfilepath,
          index=False,
          compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1}) # if a compressed version is preferred